## Classy classification

Нашу задачу можно свести к задаче классификации предложений текста между двумя классами - целевым и nontype


Classy classification - библиотека, в которой содержатся модели для классификации текстов. 

### Установка

In [1]:
!pip install classy-classification
!pip install pymorphy2
# !wget https://github.com/buriy/spacy-ru/releases/download/v2.3_beta/ru2_combined_400ks_96.zip
# !unzip ru2_combined_400ks_96.zip
# !pip install spacy==2.1.9
# !git clone -b v2.1 https://github.com/buriy/spacy-ru.git && cp -r ./spacy-ru/ru2/. /content/ru2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=f084c1f7088e8858c41bc26a2300b95b16a059f43f1a16c6504bf0cf87afe494
  Stored in directory: /root/.cache/pip/wh

In [2]:
!pip install pystemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp39-cp39-linux_x86_64.whl size=593479 sha256=31a175c9a04ee2f010fbeedb311aa451822f6aa2366a80da15a5ab786ad1a7da
  Stored in directory: /root/.cache/pip/wheels/38/b5/46/27043697131c7c21e606a0107b90c480f094c38160c0e656f7
Successfully built pystemmer


In [3]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!python -m spacy download ru_core_news_lg

2023-04-12 15:42:54.417406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 15:42:55.583716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 83.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


### Импорты

In [6]:
from utils import accuracy_score, choose_minimum_rate_sentence
from sklearn.model_selection import train_test_split
from random import randint
from pprint import pprint

import matplotlib.pyplot as plt
import classy_classification
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import spacy


In [7]:

with open('/content/contract_enforcement.pkl', 'rb') as fp:
  contract_enforcement = pickle.load(fp)
print(len(contract_enforcement))
contract_enforcement = pd.Series(contract_enforcement)

contract_enforcement = contract_enforcement[~contract_enforcement.isna()]
contract_enforcement = contract_enforcement.values
len(contract_enforcement)

988


984

In [8]:
with open('/content/garantee_enforcement.pkl', 'rb') as fp:
  garantee_enforcement = pickle.load(fp)
garantee_enforcement = pd.Series(garantee_enforcement)
print(len(garantee_enforcement))

garantee_enforcement = garantee_enforcement[~garantee_enforcement.isna()]
garantee_enforcement = garantee_enforcement.values
len(garantee_enforcement)


811


508

In [37]:
with open('/content/nonetype.pkl', 'rb') as fp:
  nonetype_sentences = pickle.load(fp)
nonetype = []

for i in range(len(nonetype_sentences)):
  for j in range(len(nonetype_sentences[i])):
    if len(nonetype_sentences[i][j]) > 5:
      nonetype.append(nonetype_sentences[i][j])

nonetype = np.array(nonetype)
len(nonetype)

11232

In [40]:
nonetype[:5]

array(['Версия с 04.07.2022 года',
       'Ю «Государственное автономное учреждение',
       "Саха (Якутия) ''Республиканская больница №1 -",
       "Национальный центр медицины''»",
       'Подписано усиленной квалифицированной электронной'], dtype='<U867')

In [41]:
df = pd.read_json('/content/train_with_splitted_texts_extended.json', encoding='utf-8')

In [42]:
df.head()

,id,text,label,target,not_found
0,809436509,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,NaN
1,854885310,"[ТРЕБОВАНИЯ, К СОДЕРЖАНИЮ ЗАЯВКИ участника зап...",обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,NaN
2,4382157,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,NaN
3,184555082,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,NaN
4,211645258,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,NaN


### построение модели ru

попробуем построить модель для "обеспечение исполнения контракта"

In [44]:
# будем использовать батчи по 100 значений для более быстрой работы
contract_enforcement_100 = np.random.choice(contract_enforcement, 100, replace=False)
nonetype_100 = np.random.choice(nonetype, 100, replace=False)



data={
    'обеспечение исполнения контракта': list(contract_enforcement_100),
    'nonetype': list(nonetype_100)}
nlp = spacy.blank('ru')
nlp.add_pipe(
    'text_categorizer',
    config={
        'data': data
    }
)

In [45]:
query = 'Рандомный текст для теста.'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 0.9997370768787411, 'nonetype': 0.0002629231212588718}


In [46]:
%%time
query = 'Размер обеспечения исполнения Контракта составляет 5 (пять) процентов цены контракта'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 0.00631879304168066, 'nonetype': 0.9936812069583195}
CPU times: user 52 ms, sys: 993 µs, total: 53 ms
Wall time: 58.1 ms


результат классификаии обратный - чем ближе текст к искомому, тем меньше значение. это связано с плохой работой русскоязычных моделей. classy classification на английском работает отлично 

In [47]:
df_contract = df[df['label'] == 'обеспечение исполнения контракта'].reset_index(drop=True)
df_contract.shape

(988, 5)

In [48]:
%%time
predictions = []

for i in range(len(df_contract)):
  result = {}
  for sentence in df_contract.loc[i, 'text']:
    result[sentence] = nlp(sentence)._.cats['обеспечение исполнения контракта']

  predictions.append(choose_minimum_rate_sentence(result)[0])


CPU times: user 35min 16s, sys: 4.16 s, total: 35min 20s
Wall time: 35min 30s


In [49]:
accuracy_score(predictions, list(df_contract.loc[:, 'target'].values))

0.5121457489878543

Видим, что c русским языком модель работает не очень стабильно, но результат accuracy 0.51 выглядит неплохо. 

Ещё один минус такой модели - долгая работа. Т.к. модель не обучается, а каждый раз считает результат на основе входных данных. 

### ru2eru_core_news_lg

In [50]:
contract_enforcement_100 = np.random.choice(contract_enforcement, 100, replace=False)
nonetype_100 = np.random.choice(nonetype, 100, replace=False)

data={
    'обеспечение исполнения контракта': list(contract_enforcement_100),
    'nonetype': list(nonetype_100)}
nlp = spacy.load('ru_core_news_lg')
nlp.add_pipe(
    'text_categorizer',
    config={
        'data': data
    }
)

In [51]:
%%time
predictions = []

for i in range(len(df_contract)):
  result = {}
  for sentence in df_contract.loc[i, 'text']:
    result[sentence] = nlp(sentence)._.cats['обеспечение исполнения контракта']

  predictions.append(choose_minimum_rate_sentence(result)[0])


CPU times: user 49min 5s, sys: 6.73 s, total: 49min 12s
Wall time: 49min 22s


In [52]:
accuracy_score(predictions, list(df_contract.loc[:, 'target'].values))


0.4888663967611336